# Synopsis

In this unit we will learn the basics of analyzing structured data. In the process we will cover:

* What is structured data
* How to use Pandas to read and write structured data
* Basic indexing operations of Pandas
* Basic operations (math and plotting) with Pandas
* Handling dates and times
* The 'split-apply-combine' framework for analysis

# Read libraries

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from colorama import Back, Fore, Style
from copy import copy, deepcopy
from pathlib import Path


In [ ]:
import datetime
import os

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import seaborn as sns

from module_libraries.my_stats import place_commas

my_fontsize = 15

# An Introduction to Pandas

Pandas is a contraction for "panel data", such as what you create when using a spreadsheet. Pandas can do a lot, so I find it easier to think of it as this for data:

<center>
    <img src = 'Images/swiss_army_knife.png', width = 500>
</center>

But first let's confront the ugly reality.

## Pandas is not very Pythonic

Actually **using** Pandas and not just using it to read files can be conceptually difficult and is a bit of a mental switch compared to most of what we have learned so far. If you want to iterate over things, you can't use a `for` loop easily. Instead you'll need to use specific Pandas methods to do whatever functions you want. These little differences add up and can wear on you, which might make you want to stop using Pandas. That's a fine way to feel (you don't really *have* to use it), but there are some big benefits to using it, that for a lot of people, are worth the costs.

## Benefits of Pandas

1. Pandas handles a lot of file I/O drudgery for you. I'll show you this in a bit, but reading CSV files and accessing data in them is super simple
2. Pandas has a lot of *magic* built into, automaticallly taking care of many type conversions after reading a file
3. Using Pandas **is** like working with [SQL](https://en.wikipedia.org/wiki/SQL) (don't know what SQL is? Don't worry, it's a bit advanced for this course but is surely something you'll encounter if you continue to program so it's worth reading up on). So learning Pandas means that you'll have a good idea of the underpinnings how SQL databases work which might help you later in your programming education (although the syntax is different).

If you like using or want to continue using Pandas here is some recommended additional reading

The Pandas tutorial pages http://pandas.pydata.org/pandas-docs/stable/tutorials.html

10 minutes to Pandas http://pandas.pydata.org/pandas-docs/stable/10min.html

## Data structures in Pandas

First, let's show some of this `automagic`. In the `Data` folder there is a folder named `College-Majors` that contains a number of files with information on career outcomes for graduates from a large collection of college majors.

This is what it looks like:

In [ ]:
cat Data/College-Majors/recent_Arts.csv

Let's load one of those files right now.

In [ ]:
data_folder = Path.cwd() / 'Data' 
college_folder = data_folder / 'College-Majors' 

arts_file = college_folder / 'recent_Arts.csv'
pd.read_csv( arts_file )

When Pandas reads in a `CSV` it turns it into its own data structure called a `DataFrame`. This `DataFrame` is actually a Python class, you can think of it as just a type of *object*. Our data is inside this *object* and it controls how we can interact with it (so you can see the first difference between this and regular programming).

(The nice formatting that makes it look like an Excel spreadsheet is provided by the `Jupyter Notebook`!)

Now, let's actually store the `CSV` data so that we can operate on it.

In [ ]:
df = pd.read_csv(arts_file)
print(type(df))

## The basics of a DataFrame

Since the object we're working with is a `DataFrame` you'll very frequently see people assign it to a variable named `df`. 

Specifying a value in a `DataFrame` requires two coordinates: a *column* and an *index*.

The **columns** run across the **top**

The **indices** run down the **left** (for now, you can think of these as rows)

We can get see the possible values for these coordinates by calling them by name from the `DataFrame`

In [ ]:
#The columns are the labels across the top
print( df.columns ) 
print()

#The indexes run down the side
print( df.index )

## Accessing data in Pandas

Pandas supports two approaches for accessing data that is stored in a **column**. 

One approach is the `.attribute` of object (think `string.whitespace`): 

In [ ]:
print(type(df.Major))
print()
df.Major

<br>

Notice that the output of using the `.attribute` approach is a different kind of object: a `Series`.

<br>


The other approach is the `key` in `dictionary`:

In [ ]:
print(type(df['Major']))
print()
df['Major']

In [ ]:
df['Major'] == df.Major

<br>

You can see that these two approaches are quite interchangeable.  However, the attribute approach is difficult to use if column names include spaces. That is one reason to name columns using `_` instead of ` `.


In [ ]:
df.Major

<br>


In order to access the information from specific 'rows', we must make use of the **indices**. 

`DataFrame` indices can be treated quite similar to the `indices` in a `list`.

In [ ]:
print( f"The first value in the 'Major' column is {df['Major'][0]}\n" )

print( f"The second value in the 'Major' column is {df.Major[1]}\n")

## Slicing in Pandas

We can also access **slices of the rows** in a `dataframe` by using the slicing syntax introduced earlier.

In [ ]:
print( df['Major'][0::2] )
print()

print(df.Major[6::-2])
print()

print( df[0::2]['Major'] )

<br>



You may also wish to **slice on columns**...


In [ ]:
print( df['Major':'Total'])

**Nah! That does not work!!!**

**We need a method based on labels in order to slice along columns.**

<br>
<br>
<br>

### `.loc[]`

The `.loc[]` approach is intended to work on labels.  

**Note that when you are using this approach you have to specify slices for both rows and columns.**

And the slices are placed within a single set of `[]`.  This is similar to how the slicing of `numpy` arrays works.

In [ ]:
print( type(df.loc[:5, 'Major':'Total']) )
print()

print( df.loc[:5, 'Major':'Total'] )

<br>

**Notice that with `.loc` the upper limits of the slice are included!!!!** 


<br>

Notice that the order of the labels matter.  

**Indices must came before column names.**

In [ ]:
print( df.loc['Major':, :] )

<br>

And `.loc[]` really wants to see two sets of labels separated by a comma.

In [ ]:
df.loc[0]

<br>

As you can see, it just assumed that we are asking for all the columns!

> `df.loc[0]` $\equiv$ `df.loc[0,:]`


In [ ]:
df.loc[0,:]

<br>
<br>
<br>


### `.iloc[]`

The `.iloc[]` approach is integer based and works on the indices.

Notice that 'Major' is the third column. Then, it follows that...




In [ ]:
print( type(df.iloc[:5, 2:5]) )
print()

print( df.iloc[:5, 2:5] )

<br>

**Notice that with `.iloc` the upper limits of the slice are not included!!!**


## How the row indices work


To gain a better understanding of how row indices work $-$ recall that they are objects of type `RangeIndex` $-$ we will do something to the `dataframe` that will change the order of the rows. 

To do this, we just have to sort the rows of the `dataframe` by the values in the **Total** column.

In [ ]:
df_total_sorted = df.sort_values('Total', ascending = False)
df_total_sorted

**Notice how the numbers in the first (unnamed) column are not in order!**

It is clear now that their values are not truly an order index.  Just like the column names, they are **labels**.

But, when we use the `.iloc` we are operating on order indices. Let us then see what slicing on `df_total_sorted` returns...

In [ ]:
print('Slicing on df\n--------------------------------------------------')
print( df.iloc[:4, 2:5] )
print()

print('\nSlicing on df_total_sorted\n--------------------------------------------------')
print( df_total_sorted.iloc[:4, 2:5] )
print()


What we get with `iloc` is what we also get if we use the attribute approach or the `key` and `dictionary` approach.


In [ ]:
print('Slicing on df\n--------------------------------------------------')
print( df.Major[:4] )
print()

print('\nSlicing on df_total_sorted\n--------------------------------------------------')
print( df_total_sorted.Major[:4] )
print()

print('\nSlicing on df_total_sorted\n--------------------------------------------------')
print( df_total_sorted['Major'][:4] )
print()

<br><br>


Because the indices are actually labels, when we use `.loc[]` on `df_total_sorted` we should expect to obtain something quite different from before...

In [ ]:
print('Slicing on df\n--------------------------------------------------')
print( df.loc[:4, 'Major':'Total'] )
print()

print('\nSlicing on df_total_sorted\n--------------------------------------------------')
print( df_total_sorted.loc[:4, 'Major':'Total'] )
print()

# `Series`: Pandas' other data type

You will recall that earlier on when we retrieved the values from a single column, `pandas` returned a `Series` object.

We can create `Series` by pulling a single column or a single row from a `dataframe.



In [ ]:
print('---- Single column\n')
serie_1 = df['Major']
print(serie_1)

print('\n\n---- Single row\n')
serie_2= df_total_sorted.loc[3]
print(serie_2)

<br><br>

Many of the methods defined on `dataframes` also work on `Series` as long you account for the fact that there is only one column, so you do not need a label for it.

In [ ]:
print( serie_1[0:6:2] )
print()

print( serie_2[0:6:2] )
print()

print( serie_2['Major':'Total'] )

# The benefits of using `pandas`

Pandas has a LOT of cool functionality. It can read (and write) `.xls`/`.xlsx` files! 

**Now you don't need to open a workbook in Excel and save it to a `CSV` when someone sends them to you!**

When we read an Excel spreadsheet, all we have to say is what sheet we want to use in the file. You can use either the sheet_name (if it has one) or just give it the index of the sheet.

In [ ]:
df_arts = pd.read_excel( college_folder / 'recent_Arts.xlsx', sheet_name = 0)
df_arts


<br>

If you want to know how many sheets there are in the *Excel* workbook, you can use `.ExcelFile()` instead

In [ ]:
with pd.ExcelFile( data_folder / 'recent_Arts.xlsx' ) as reader:
    line = ''
    for name in reader.sheet_names:
        line += f"\t{name}\n" 
    print(f"This excel file has {len(reader.sheet_names)} worksheets.\n"
          f"Their names are:\n{line}")


And it's not just *Excel*. `Pandas` can actually read/write a large number of different and really useful file formats that can be essential when working with collaborators who might not be quite so `Python` inclined:

> read_clipboard
>
> read_csv
>
> read_excel
>
> read_feather
>
> read_fwf
>
> read_gbq
>
> read_hdf
>
> read_html
>
> read_json
> 
> read_orc
>
> read_parquet
>
> read_pickle
>
> read_sas
>
> read_spss
>
> read_sql
>
> read_sql_query
>
> read_sql_table
>
> read_stata
>
> read_table
>
> read_xml
        

## Visualization

Pandas helps you quickly explore and manipulate data as you learn about your dataset basics. One quick benefit is the built-in plotting directly from the dataframe.

Let's say that we wanted to make a plot that examined the difference between the majors in terms of the raw employment numbers.

In [ ]:
sns.set_context("talk")
sns.set_style('darkgrid')

In [ ]:
df_arts['Employed'].plot();

<br>

Okay, so it's pretty ugly, but it was also fast and easy! Note that the `;` suppresses `matplotlib` print statements

Let's actually think about this for a moment because something magical just happened and we all probably took it for granted.

**We just plotted numeric data from the file that I read in with a single command.**

**When did I change the type of that data to be an integer so that we could plot it??**

You might recall when we read files in the past using `open('super_cool_file.csv')`, **everything was read as a string by default, even numbers!**

When we load data with Pandas it automatically converted the 'Employed' column data to integers. In fact, Pandas does this with all of the columns and when it does this it picks the **least** expansive data type that **accommodates all the data in the column**.

We can check this, so the `Unemployed` column should be integers also.

In [ ]:
df_arts['Unemployed']

You can see that it is actually `int64` which is the data format used by `numpy`.

Following on this success, we can expect the `ShareWomen` column to contain `float64`.

In [ ]:
df_arts['ShareWomen']

<br>

Something to be aware of though, is that if we had a single string in that column of data **none of it would be converted**. 

All of the read values would be strings because any number can represented as a string, just as text data can.

<br><br>


**Getting back to our goal of plotting the data.**

A bar plot is more appropriate for this data...

In [ ]:
df['Employed'].plot(kind = 'bar');

<br> Ah! That's a little bit better!

But we should never have a graph without a y-label! To change labels we'll need to operate on the `matplotlib` object. 

**Applying the method `plot()` to a `DataFrame` returns a `Matplotlib` axis object.**

We can then use what we know about `Matplotlib` to customize the graph.

In [ ]:
# I'm changing the color of the bars!
#
ax = df['Employed'].plot(kind = 'bar', color = 'darkred')

# Now I can set the y-axis label
#
ax.set_ylabel('Students Employed', fontsize = my_fontsize * 1.5)
ax.set_yticks(range(0, 100000, 20000))
y_labels = [place_commas(x) for x in range(0, 100000, 20000)]
ax.set_yticklabels(y_labels)

# I can also set the xticks to the major names. And 
#
major_labels = df['Major']
x_labels = [x.capitalize() for x in major_labels]
ax.set_xticklabels(x_labels, rotation = 45, ha = 'right')

plt.show()

## Mathematical operations

Pandas has methods implementing numerous mathematical operations built directly into the `dataframe` object using `numpy`. 

For exampple, if you want to know the average of a column's value or the number of rows with entries (not every position has to have a value!), you can accomplish that in a straightforward manner.

Let's start by just counting the number of values in each column (it should be 8 in every column since every spot in our spreadsheet was filled out).

In [ ]:
print(df['Employed'].count())
print()

df['Employed']

In [ ]:
help(df.count)

You can see in the documentation that

>Count non-NA cells for each column or row.
>    
>    The values `None`, `NaN`, `NaT`, and optionally `numpy.inf` (depending
    on `pandas.options.mode.use_inf_as_na`) are considered NA.


When there is a missing value in the raw data `pandas` replaces that value with a `Not a Number` or `NaN`.

We can learn about what this means by creating a new column with no values.

In [ ]:
df['Test_column'] = np.nan
df.loc[:, 'College_jobs':]

Now when we count, it won't be the same as before.

In [ ]:
df['Test_column'].count()

We can also count values for the entire `dataframe`

In [ ]:
df.count()

<br>

We can also just count a few columns or single column by chaining the `.count()` method after we slice the `dataframe`.

In [ ]:
df.loc[0:2].count()

<br><br>

There are other useful functions built in too. We can quickly take the mean or median of a column also.

In [ ]:
df.Employed.mean()

In [ ]:
df.Employed.median()

Or we can even get the mean, medians, and a host of other summary statistics for all columns!

In [ ]:
df.Employed.describe()

<br>

As you would expect, the `.describe()` method returns a `Series`

In [ ]:
print( type( df.Employed.describe() ) )
print()

df.Employed.describe()['mean']

<br><br>




Thus, it can be accessed in the usual ways:

In [ ]:
my_column = 'Men'
results = df[my_column].describe()

print(results['50%'])
print()

print(results.iloc[5])
print()

print(f"The median of number of {my_column} is {results['50%']}")

## Easy "parallelization" of  code

Just as with `numpy`, `pandas` enable us to easily parallelize operations on columns of data. 

Note that the data types with a single row are unlikely to be identical, so the similar approach for rows would be unlike to work.

You can easily implement operations across all values in a column. For example, you can divide the values in a column by the corresponding values in another column.

In [ ]:
df['Employed']/df['Total']

If you want to save the outcome of this operation, you assign it to another variable or to a new column in the `dataframe`.

In [ ]:
df['Percent_Employed'] = 100 * df.Employed / df.Total

We can use the `.head()` method to print the top few rows of the `DataFrame`

In [ ]:
df.loc[:, 'College_jobs':].head(3)

<br><br>

And you can perform more elaborate calculations too... the z-score is given by:

In [ ]:
df['z_score_college_jobs'] = ( (df.College_jobs - df.College_jobs.mean()) / 
                                df.College_jobs.std() )

df['z_score_college_jobs'] 

## Querying

A great strength of Pandas is the ability to query a `dataframe` and extract only the rows or columns that meet some criteria.

For example, imagine you want to focus on those majors with more than 20,000 graduates.

In [ ]:
df.Total

The following statement parallelizes the comparison of the values in the column *Total* to the value 20,000 

In [ ]:
df.Total > 20000

This Boolean `Series` can be used to filter the appropriate rows in the `dataframe`

In [ ]:
df[df.Total > 20000]

This is where things get annoying.  `pandas` developers chose, for convenience, I imagine, to settle on a distinct set of symbols for the logical operator:

> `|` stands for `or`
>
>  `&` stands for `and`
>
> `~` stands for `not`

And notice that **every individual logical operation must be placed inside parentheses**.

In [ ]:
df[(df.Total > 20000) | (df.Major_code < 6003)]

In [ ]:
df[(df.Total > 20000) & (df.Major_code < 6003)]

We can chain selection operators off of the query also if we want to know a bit more about the resulting column

In [ ]:
df[df.Employed > 20000].Unemployment_rate.mean()

<br><br>

Querying is actually a sort of parallelization. It enables us to very compactly tell the computer what operations to implement across all rows or columns. The masks returned by queries could be easily implemented with `for` loops

    for total_students in total:
        if total_students > 20000:
            #Continue with code
            
However, a large set of indented `if` statements and of `loops` can quickly become unyielding. 

Conversely, the terseness of `Pandas`' notation can also become an hindrance to readability and a source of logical mistakes. To avoid mistakes, **it is good to construct your masks step-by-step and to carefully test how accurate they are at capturing your intent**. 

## Working with dates

We talked earlier about how Pandas automagically converts data types when reading a file in. One of the best automagic features is `pandas`' ability to handle dates and times. 

In order to get a better idea of what Pandas do, it is good to compare with the `datetime`.   `datetime` is complex, sophisticated library that can be a bit unfriendly. For example, the module one typically uses the most has the exact same name as the package itself:

Using `import datetime`, one would refer to `datetime.datetime`

Using `from datetime import datetime` or `import datetime.datetime as datetime`, one would refer to the same modules simply as `datetime`

Earlier, we used `import datetime`.

This package has many useful methods and attributes. For example, 


In [ ]:
datetime.date.today()

If we want just the `date` we can use the `date` module in datetime.

But typically most people care about what time it is too. To get both the date and the time, we must use the `datetime` module

In [ ]:
datetime.datetime.today()

So that's how dates and times are generated! The benefit is that the time is returned in a `datetime` object so we can access individual parts of the time by name, like so:

In [ ]:
today = datetime.datetime.today()

print(f"Today's year is {today.year}")
print(f"Today's month is {today.month}")
print(f"Today's day is {today.second}")

<br><br>

But how do we read in a list of dates? For that we actually have to convert a string using the `datetime.datetime.strptime` function. This function takes two arguments:

    1. the string we want to decode
    2. a string providing the format of the date we want to decode 
    
The formating string uses a special [symbol set](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior).


In [ ]:
datetime.datetime.strptime('2014-01-01', '%Y-%m-%d')

<br><br>

Sadly, different data sources use different formating for dates and times. Think of the European versus US styles for dates. 

Fortunately, Pandas can take care of these issues for us!

In [ ]:
aapl_df = pd.read_csv( Path.cwd() / 'Data' / 'aapl_stock_price.csv' )

aapl_df.head(10)

In [ ]:
aapl_df.Date.head()

It automatically read in the dates! It understands time and can use basic functions with the column

In [ ]:
aapl_df.Date.max()

In [ ]:
aapl_df.Date.min()

And how do we know `Pandas` truly processes these values as true dates? 

Well, if it did, it should be able to filter data bases on year alone, right?

In [ ]:
aapl_df[aapl_df.Date > '2013']

<br><br>

It sure looks like it does... **but does it really?**

In [ ]:
print(aapl_df.Date[0])
aapl_df.Date[0].month

**No, it does not!!!**

Because these values are not really ingested as dates, we can't access individual attributes of a date.

<br><br>

Nonetheless, as long as the dates being read are in a guessable format, it is able to 'fake' it under many situations, including sorting the data in the `dataframe` by 'date'. 

In [ ]:
aapl_df.Date.sort_values()

Of course, what `pandas` is actually doing with these value is sorting them as strings!


**If we need real `datetime` objects  though, `pandas` has a function to convert it.**

In [ ]:
aapl_df['dt_Date'] = pd.to_datetime(aapl_df.Date)
aapl_df.head()

<br>
Every individual value is a `datetime.date` object

In [ ]:
aapl_df.dt_Date[0].day

In [ ]:
aapl_df.dt_Date.day

So, sadly, we cannot extract the day from every row in the column.


This means that we even when using the `datetime` transformation, we are pretty much stuck having to make comparisons based on string format. 

That is, this works...

In [ ]:
aapl_df[aapl_df.dt_Date < '1981' ]

<br><br><br>

but this does not...

In [ ]:
aapl_df[aapl_df.dt_Date == '1980' ]

<br><br>
### Dates as indices
**Another cool feature is Pandas' ability to plot time series and referring directly to dates! You just have to use the column with the `datetime` objects as the index column.**

In [ ]:
aapl_df.set_index('dt_Date', inplace=True)

In [ ]:
ax = aapl_df.Close.plot(color = 'darkred')

ax.set_ylabel('Closing Price\n(USD$)', fontsize = 1.3*my_fontsize)
ax.set_xlabel('Date', fontsize = 1.3*my_fontsize);

## Helpful built-in functions

`Pandas` has a number of built-in methods that help with the analysis of panel data.  For example, `.rolling()` allows for the construction of analysis windows that move along an axis and include a specified number of values. 


In [ ]:
#There are roughly 260 workdays in a year
#
close_rolling_window = aapl_df['Close'].rolling(260, center = True)
print(type(close_rolling_window))
print()


close_rolling_ave = close_rolling_window.mean() 
print(type(close_rolling_ave))
print()


# Now we just plot it
#
ax = aapl_df.Close.plot( color = 'darkred', fontsize = my_fontsize)
close_rolling_ave.plot( label = 'Moving Average', color='steelblue', lw = 8, 
                        alpha = 0.5)

ax.set_ylabel('Closing Price\n(USD$)', fontsize = 1.3*my_fontsize)
ax.set_xlabel('Date', fontsize = 1.3*my_fontsize);



<br><br>

What if we were interested in the movement of the stock between each day (i.e. did the stock lose or gain money from one day to another)?  We can do that easily with the `diff()` function.

In [ ]:
close_daily_diff = aapl_df['Close'].diff()

ax = close_daily_diff.plot( color = 'darkred', fontsize = my_fontsize )

ax.set_ylabel('Daily Return\n(USD$)', fontsize = 1.3*my_fontsize)
ax.set_xlabel('Date', fontsize = 1.3*my_fontsize);


<br><br>

And remember at anytime we can restrict the dataset with a query and plot only that portion

In [ ]:
close_daily_diff = aapl_df[aapl_df.Date < '2012']['Close'].diff()

ax = close_daily_diff.plot( color = 'darkred', fontsize = my_fontsize )

ax.set_ylabel('Daily Return\n(USD$)', fontsize = 1.3*my_fontsize)
ax.set_xlabel('Date', fontsize = 1.3*my_fontsize);


<br><br>

Some users of `Pandas` like to **chain** a lot of operations. 

**This is not Pythonic!** If done in excess, it makes code harder to read and debug.  When re-factoring make sure to de-convolute your code.


In [ ]:
aapl_df[aapl_df.Date < '2012'].Close.diff().mean()

In [ ]:
aapl_df[aapl_df.Date < '2012'].Close.diff().abs().hist(bins=60, color='darkred', 
                                                       log=True);

# `Pandas` versus dictionaries

`Pandas` is really useful for loading data and for the parallelization of some operations. However, some of me sometimes prefers to work with dictionaries.

You can cast the data within a row, a column or an entire `dataframe`.  

Helpfully, the attribute `.values` returns a `numpy` array.


In [ ]:
totals = df.Total.values

print(type(totals))
print()
print(totals)

You can also dump the data into a `list`

In [ ]:
list_totals = df.Total.tolist()

print(type(list_totals))
print()
print(list_totals)

Had any of the manipulations we performed here actually been important, we'd of course want to save them. 

With `pandas`, you can easily export your data in a number of formats. 

In [ ]:
# place cursor after underscore and press Tab
#
df.to_

As an example, consider

In [ ]:
df.to_excel( college_folder / 'recent_Arts_edited.xlsx', sheet_name = 'Arts', 
             index = False )

You can check that the file was created...

In [ ]:
os.listdir(college_folder)

# Exercises

Let's look at **all fields** now.  

Read the data from the corresponding `.csv` or `.xlsx` file into a `dictionary` of `dataframes` where the `key` is the name of the field.

Write out a nice table with a count of the number of majors for each field

Find out the major within each field with the highest unemployment rate

Calculate the average unweighted unemployment rate of all the majors in each of the fields 

Is this unweighted average representative of what is experienced by the typical graduate in the field?

Calculate the weighted average unemployment rate in each of the fields 

# Let's keep going...

[Next lesson](nb_09_Discrete_random_variables.ipynb)